In [ ]:
!pip install TwitterAPI

"""
Summarize data.
"""
import json
import networkx as nx
def read_file(filename):
    """
    Reading data from file
    """
    array = []
    with open(filename) as f:
        for line in f:
            array.append(json.loads(line))
    return array
    pass

def read_txt(filename):
    """
    Reading data from file
    """
    array = []
    with open(filename) as f:
        for line in f:
            array.append(line)
    return array
    pass


def main():
	file_collect = read_file("classify_data.json")
	file_friends = read_file("classify_friend.json")
	file_cluster = read_txt("cluster_data.txt")
	file_classify = read_file("sentiment_data.json")
	print("\n Number of users collected:", len(file_friends))
	print("\n Number of messages collected:",len(file_collect))
	print("\nNumber of communities discovered:", len(file_cluster))
	for i in range(0,len(file_cluster)):
		print("Cluster ", i , " number of elements " , len(file_cluster[i]))
	negative_tweets = []
	positive_tweets = []
	balanced_tweets = []
	for u in file_classify:
		for i in u:
			if i['sentiment'] =="negative":
				negative_tweets.append(i)
			elif i["sentiment"] == "positive":
				positive_tweets.append(i)
	print("Number of instances per class", len(file_collect))
	print("Tweets \n The negative tweets are: ",len(negative_tweets), "\n The positive tweets are: ", len(positive_tweets), "\n The balanced tweets are: ",len(balanced_tweets))
 	#print("Number of instances per class found ",len(file_collect))
 	#print(" tweets \n The negative tweets are: ",len(negative_tweets), "\n The positive tweets are: ", len(positive_tweets), "\n The balanced tweets are: ",len(balanced_tweets))
	print("One example from each class:")
	print("negative:", negative_tweets[0])
	print("\n\npositive:", positive_tweets[0])
	pass

if __name__ == "__main__":
    main()



"""
collect.py
"""

from collections import Counter, defaultdict
from TwitterAPI import TwitterAPI
import time
import pickle
import signal
from threading import Thread, Condition
from multiprocessing import Queue
import sys
import json
import threading
import numpy as np

queueSize = 500
queue = Queue(queueSize)
twitter_rate_limit = Condition()
#Filename
CLASSIFY_FILE_NAME= "classify_data.json"
CLASSIFY_FRIEND_NAME= "classify_friend.json"

# Service Tokens
consumer_key = 'ftwgjnC4SR8U4lEPNHqNKiNCV'
consumer_secret = 'DiIXS7Z0NkrYgtes9zbD666AwhJiC9AM4lBTuv8aMrKOAb7jBC'
access_token = '596398510-mYstiQ7CekWvoqXsDKdi7CxxaRgiGlTsgsnigT5z'
access_token_secret = 'i5a2c5379p1UCCYbJfLnkEdIycDRuR9YvjLvzHCxSA410'


class CreateData(Thread):
    def __init__(self, run_event, service, keywords,file):
        self.run_event = run_event
        self.service = service
        self.comments = self.robust_request(self.service,'statuses/filter', {'language': 'en', 'track': keywords})
        self.file = file
        Thread.__init__(self)
        
    def run(self):
        global queue
        for comment in self.comments:
            if not self.run_event.is_set():
                break
            try:
                print(comment['text'])
            except:
                continue
            queue.put(comment)
            data = {}
            data["tweet"] = comment["text"]
            data["created_at"] = comment["created_at"]
            data["user_name"] = comment['user']["screen_name"]
            data["user_id"] = str(comment['user']["id"])
            data["location"] = comment['user']["location"]
            json.dump(data, self.file)
            self.file.write("\n")
            
    def robust_request(self, twitter, resource, params, max_tries=5):
        for i in range(max_tries):
            request = twitter.request(resource, params)
            if request.status_code == 200:
                return request
            else:
                print('Got error %s \nsleeping for 15 minutes.' % request.text)
                sys.stderr.flush()
                twitter_rate_limit.acquire()
                twitter_rate_limit.wait(61 * 15)
                twitter_rate_limit.release()
                if not self.run_event.is_set():
                    break

class CollectFriendData(Thread):
    def __init__(self, run_event, service, keywords, file):
        self.run_event = run_event
        self.service = service
        self.users_file = file
        Thread.__init__(self)
        
    def run(self):
        global queue
        while self.run_event.is_set():
            comment = queue.get()
            user_id = comment['user']['id']
            request = self.robust_request(self.service, "friends/ids", {'user_id': user_id})
            if request is None:
                continue
            data = {}
            data["user_id"] = str(comment['user']["id"])
            data["user_name"] = comment['user']["screen_name"]
            data["friends"] = list(request)
            json.dump(data, self.users_file)
            self.users_file.write("\n")      
            
            
    def robust_request(self, twitter, resource, params, max_tries=5):
        for i in range(max_tries):
            request = twitter.request(resource, params)
            if request.status_code == 200:
                return request
            else:
                print('Got error %s \nsleeping for 15 minutes.' % request.text)
                sys.stderr.flush()
                twitter_rate_limit.acquire()
                twitter_rate_limit.wait(61 * 15)
                twitter_rate_limit.release()
                if not self.run_event.is_set():
                    break

def yes_or_no(question):
    reply = str(input(question+' (y/n): ')).lower().strip()
    if reply[0] == 'y':
        x = input("Please keywords you want to have use for analysis in one line using commas\n (Recommanded related keywords for better search) \n").split(',')
        keywords = np.asarray(x)
        return keywords
    if reply[0] == 'n':
        return True 
    else:
        return yes_or_no("Uhhhh... please enter y or n ")
pass

def main():
    run_event = threading.Event()
    run_event.set()
    service = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)
    file1 = open(CLASSIFY_FILE_NAME, "w")
    file2 = open(CLASSIFY_FRIEND_NAME,"w")
    keywords = ["Avengers End Game", "Avengers", "End Game", "MCU" , "Marvel", "Marvel Cinematic Universe", "Iron Man","Thanos", "Hulk"]
    answer = yes_or_no("This Analysis shows results for 'Avengers:EndGame' movie, Do you want to try something else?")
    if(isinstance(answer,np.ndarray)):
        keywords = answer
    collect_comments = CreateData(run_event, service, keywords,file1)
    collect_friends = CollectFriendData(run_event, service, keywords, file2)
    try:
        collect_comments.start()
        collect_friends.start()
    except:
        run_event.clear()
        twitter_rate_limit.acquire()
        twitter_rate_limit.notify()
        twitter_rate_limit.release()
        collect_comments.join()
        collect_friends.join()
        file1.close()
        file2.close()
        sys.exit()
        quit()
    try:    
        while queue.qsize() < queueSize:
            time.sleep(.1)
        raise Exception("Exit")
    except Exception:
        print("Data Collected! Exiting")
        run_event.clear()
        twitter_rate_limit.acquire()
        twitter_rate_limit.notify()
        twitter_rate_limit.release()
        collect_comments.join()
        collect_friends.join()
        file1.close()
        file2.close()
        sys.exit()
        quit()
       
    
if __name__ == '__main__':
    main()
    
"""
Classify data.
"""
import json
import re
from collections import Counter
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

CLASSIFY_FILE_NAME= "classify_data.json"

def read_file(filename):
    """
    Reading data from file
    """
    array = []
    with open(filename) as f:
        for line in f:
            array.append(json.loads(line))
    return array
    pass

def get_tokens(tweets):
    
    tokens = []
    for tweet in tweets:
        tweet["token"] = []
        text = tweet['tweet'].lower()
        text = re.sub('@\S+', ' ', text)  # Remove mentions.
        text = re.sub('http\S+', ' ', text)  # Remove urls.
        token = re.findall('[A-Za-z]+', text)
        tokens.append(token) # Retain words.
        #print(token)
        tweet["token"] = token
    return tokens
    pass

def purne_token(tweets, wordCount):
    word_counts = Counter()
    for tweet in tweets:
        word_counts.update(tweet["token"])
    # Retain in vocabulary words occurring more than wordCount.
    vocab = set([w for w, c in word_counts.items() if c > wordCount])
    newtoks = []
    for i, tweet in enumerate(tweets):
        newtok = [token for token in tweet["token"] if token in vocab]
        if len(newtok) > 0:
            newtoks.append(newtok)
           
    return newtoks
    pass

def download_afinn():
    url = urlopen('http://www2.compute.dtu.dk/~faan/data/AFINN.zip')
    zipfile = ZipFile(BytesIO(url.read()))
    afinn_file = zipfile.open('AFINN/AFINN-111.txt')
    afinn = dict()

    for line in afinn_file:
        parts = line.strip().split()
        if len(parts) == 2:
            afinn[parts[0].decode("utf-8")] = int(parts[1])
    print("read %d AFINN terms." %(len(afinn)))
    return afinn

def afinn_posneg(terms, afinn, verbose=False):
    pos = 0
    neg = 0
    for t in terms:
        if t in afinn:
            if verbose:
                print('\t%s=%d' % (t, afinn[t]))
            if afinn[t] > 0:
                pos += afinn[t]
            else:
                neg += -1 * afinn[t]
    return pos, neg

def afinn_sentiment(tweets, afinn):
    for u in tweets:
            u['pos'], u['neg'] = afinn_posneg(u['token'], afinn) 
            sentiment = "balanced"
            if u['pos'] < u['neg']:
                sentiment = "negative"
            else:
                sentiment = "positive"
            u['sentiment'] = sentiment
    pass

def main():
    tweets = read_file(CLASSIFY_FILE_NAME)
    print("Tweets has been read!")
    tokens = get_tokens(tweets)
    purned_tokens = purne_token(tweets,2)
    afinn = download_afinn()
    afinn_sentiment(tweets, afinn)
    negative_tweets = [u for u in tweets if u['sentiment'] == "negative"]
    positive_tweets = [u for u in tweets if u['sentiment'] == "positive"]
    print("In total ",len(tweets)," tweets \n The negative tweets are: ",len(negative_tweets), "\n The positive tweets are: ",len(positive_tweets))
    file = open("sentiment_data.json","w")  
    json.dump(tweets,file)
    file.close()
    pass

if __name__ == "__main__":
    main()    
"""
Cluster data.
"""
import json
from collections import Counter, defaultdict, deque
import copy
from itertools import combinations
import math
import networkx as nx
import urllib.request
import matplotlib.pyplot as plt
import community
import sys
import glob
import os
import time
import community
from sklearn.cluster import spectral_clustering
import scipy

CLASSIFY_FILE_NAME= "classify_friend.json"

def read_file(filename):
    """
    Reading data from file
    """
    array = []
    with open(filename) as f:
        for line in f:
            array.append(json.loads(line))
    return array
    pass
        
def build_graph(users):
    G = nx.Graph()
    for u in users:
        G.add_edges_from([str(u['user_id']), str(fid)] for fid in u['friends'])
    return G

def draw_network(graph, users, filename):
    graph_labels=dict()
    for u in users:
        graph_labels[str(u['user_id'])]=u['user_name']
    fig = plt.figure()
    plt.axis('off')
    nx.draw_networkx(graph,pos=nx.spring_layout(graph),with_labels = True, labels=graph_labels, node_size = 70, width = 0.2)
    fig.savefig(filename, format='png')
    plt.draw() 
    pass

def bfs(graph, root):
    node2distances = defaultdict(int)
    node2num_paths = defaultdict(int)
    node2parents = defaultdict(list)

    nodequeue = deque([root])
    visited = set()
    visited.add(root)

    node2distances[root] = 0
    node2num_paths[root] = 1
    depth = 0
    parent = root
    while (len(nodequeue) > 0):
        node = "".join(nodequeue.popleft())
        depth = node2distances[node] + 1
        for n in graph.neighbors(node):
            if (n not in visited):
                nodequeue.append ([n])
                node2distances[n] = depth
                node2parents[n].append(node)
                node2num_paths[n] = 1
                visited.add(n)
            elif (node2distances[n] == depth):
                node2num_paths[n] += 1
                node2parents[n].append(node)
              
    return dict(sorted(node2distances.items())), dict(sorted(node2num_paths.items())), dict(sorted((_node, sorted(_parents)) for _node, _parents in node2parents.items()))
    pass

def bottom_up(root, node2distances, node2num_paths, node2parents):
    result = dict()
    cnt = Counter()
    for n in sorted(node2distances, key=node2distances.get, reverse=True):
        if n != root:
            if node2num_paths[n] > 1:
                for a in node2parents[n]:
                    finalTuple = tuple(sorted([n, a]))
                    result[finalTuple]=(float)((cnt[n]+1)/len(node2parents[n]))
                    cnt[a]=(float)(cnt[a]+result[finalTuple])
            else:
                l = tuple(sorted([n,node2parents[n][0]]))
                result[l] = float(cnt[n] + 1) 
                cnt[node2parents[n][0]] = (float)(cnt[node2parents[n][0]]+result[l])
    
    return result
    pass

def approximate_betweenness(graph):
    betweenness = dict()
    for n in graph.nodes():
        node2distances, node2num_paths, node2parents = bfs(graph, n)
        botup = bottom_up(n, node2distances, node2num_paths, node2parents)
        for b in botup:
            if b in betweenness:
                betweenness[b] = betweenness[b] + botup[b]
            else:
                betweenness[b] = botup[b]
    for a,b in betweenness.items():
        betweenness[a]=b/2

    return betweenness
    pass

def partition_girvan_newman(graph):
    index=0
    gcopy=graph.copy()
    apxbtw = approximate_betweenness(gcopy)    
    result=sorted(apxbtw.items(),key=lambda x: (-x[1], x[0]))
    compnent =[c for c in nx.connected_component_subgraphs(gcopy)]
    while (True):
        gcopy.remove_edge(*(result[index][0]))
        compnent = [c for c in nx.connected_component_subgraphs(gcopy)]
        if len(compnent) > 1:
            break
        elif len(compnent) == 1:
            index += 1
    return compnent
    pass



def main():
    users = read_file(CLASSIFY_FILE_NAME)
    print("File has been read!")
    graph = build_graph(users)
    print("Graph in progress..... This might take a while...")
    draw_network(graph,users,"cluster_img.png")
    print("Graph Stored in cluster_img.png")
    clusters = partition_girvan_newman(graph)
    print('%d clusters' % len(clusters))
    print('first partition:');
    for i in range(0,len(clusters)):
        print("cluster ",i," has  ",clusters[i].order(),"  nodes")      
    file = open("cluster_data.txt","w")  
    file.write(str(clusters))
    file.close()
    pass

if __name__ == "__main__":
    main()

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.



 Number of users collected: 0

 Number of messages collected: 0

Number of communities discovered: 1
Cluster  0  number of elements  183
Number of instances per class 0
Tweets 
 The negative tweets are:  85 
 The positive tweets are:  431 
 The balanced tweets are:  0
One example from each class:
negative: {'tweet': "Avengers End Game made me cry 3 times during the movie and now it's making me cry a 4th time just thinking about it in bed cool", 'created_at': 'Fri Apr 26 05:51:31 +0000 2019', 'user_name': 'MaddiePolonus', 'user_id': '730074709', 'location': 'Mizzou', 'token': ['avengers', 'end', 'game', 'made', 'me', 'cry', 'times', 'during', 'the', 'movie', 'and', 'now', 'it', 's', 'making', 'me', 'cry', 'a', 'th', 'time', 'just', 'thinking', 'about', 'it', 'in', 'bed', 'cool'], 'pos': 1, 'neg': 2, 'sentiment': 'negative'}


positive: {'tweet': "RT @fineglorysht4: For all the MCU fans out there. Let's stand and listen to our National Anthem. \n\n#AvengersEndgame https://t.co/oDPX1OHII

In [ ]:

import os
cwd = os.getcwd()
cwd

In [ ]:
!pip install TwitterAPI